This notebook provides examples on how to deal with missing values using 4 different techniques: 
- Impute with zero
- Impute with mean/median/most frequent value
- Impute with K-NN or multiple imputations
- Impute with deep learning

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from math import sqrt

import random
import numpy as np
random.seed(0)

import pandas as pd
import missingno as msno 

Preprocessing

In [ ]:
# Test dataset
dataset = fetch_california_housing()
train, target = pd.DataFrame(dataset.data), pd.DataFrame(dataset.target)
train.columns = ['0','1','2','3','4','5','6','7']
train.insert(loc=len(train.columns), column='target', value=target)

In [ ]:
#Randomly replace 40% of the first column with NaN values
column = train['0']
print(column.size)
missing_pct = int(column.size * 0.4)
i = [random.choice(range(column.shape[0])) for _ in range(missing_pct)]
column[i] = np.NaN
print(column.shape[0])

20640
20640


In [ ]:
train.head()

,0,1,2,3,4,5,6,7,target
0,NaN,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,NaN,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,NaN,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


# Missing values

## Visualize

In [ ]:
# Visualize missing values per columns
msno.heatmap(df) 

## Impute zero values

In [ ]:
train.fillna(0)

,0,1,2,3,4,5,6,7,target
0,0.0000,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,0.0000,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,0.0000,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,0.0000,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


## Impute mean/median/most frequent value with Scikit

In [ ]:
# Imput mean or median function
from sklearn.impute import SimpleImputer

def imput_nan_SL(df, x): #x can be 'mean' or 'median' or 'most_frequent'
    df_corr = df.copy()
    imp_mean = SimpleImputer(strategy=x)
    mask = df_corr.columns
    df_corr[mask] = imp_mean.fit_transform(df_corr[mask])
   
    return df_corr

In [ ]:
%%time
imput_nan_SL(train,'mean')

CPU times: user 9.13 ms, sys: 2.09 ms, total: 11.2 ms
Wall time: 9.49 ms


,0,1,2,3,4,5,6,7,target
0,3.87794,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.30140,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,3.87794,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.64310,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.87794,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.56030,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.55680,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.70000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,3.87794,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


## Impute with K-NN or multiple imputations with Impyute

In [ ]:
import sys
from impyute.imputation.cs import fast_knn, mice
sys.setrecursionlimit(100000) #Increase the recursion limit of the OS

def imput_nan_knn(df,n): #x is the number of neighbors
    df_corr = df.copy()
    mask = df_corr.columns
    df_corr[mask] = fast_knn(df_corr[mask].values,n)
    
    return df_corr

In [ ]:
%%time
imput_nan_knn(train,30)

CPU times: user 7.29 s, sys: 48.3 ms, total: 7.33 s
Wall time: 7.39 s


,0,1,2,3,4,5,6,7,target
0,8.325200,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.301400,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.257400,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,3.669610,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.846200,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,3.367019,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.556800,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.700000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.867200,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


In [ ]:
import sys
from impyute.imputation.cs import fast_knn, mice
sys.setrecursionlimit(100000) #Increase the recursion limit of the OS

def imput_nan_mice(df): #x is the number of neighbors
    df_corr = df.copy()
    mask = df_corr.columns
    df_corr[mask] = mice(df_corr[mask].values)
    
    return df_corr

In [ ]:
%%time
imput_nan_mice(train)

CPU times: user 5.77 s, sys: 71.4 ms, total: 5.84 s
Wall time: 5.65 s


,0,1,2,3,4,5,6,7,target
0,8.325200,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.301400,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.257400,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,4.925636,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.846200,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,2.083081,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.556800,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.700000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.867200,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


## Impute with deep learning

In [ ]:
mask = train.drop(columns='0').columns.values
mask

array(['1', '2', '3', '4', '5', '6', '7', 'target'], dtype=object)

In [ ]:
import datawig

def imput_nan_DL(df,col,n):
    df_corr = df.copy()
    mask = df.columns
    info = df_corr.drop(columns=col).columns.values # column(s) containing information about the column we want to impute
    output_path = 'imputer_model' # stores model data and metrics
    df_train, df_test = datawig.utils.random_split(df)
    
    imputer = datawig.SimpleImputer(info, col, output_path)
    imputer.fit(train_df=df_train[mask], num_epochs=n)
    imputed = imputer.predict(df_test[mask])
    
    return imputed

In [ ]:
%%time
imput_nan_DL(train,'0',50)

2020-01-02 11:27:30,369 [INFO]  
========== start: fit model
2020-01-02 11:27:30,371 [WARNING]  Already bound, ignoring bind()
2020-01-02 11:27:30,708 [INFO]  Epoch[0] Batch [0-465]	Speed: 22384.27 samples/sec	cross-entropy=8.889659	0-accuracy=0.000000
2020-01-02 11:27:31,021 [INFO]  Epoch[0] Train-cross-entropy=9.074508
2020-01-02 11:27:31,022 [INFO]  Epoch[0] Train-0-accuracy=0.000000
2020-01-02 11:27:31,023 [INFO]  Epoch[0] Time cost=0.650
2020-01-02 11:27:31,029 [INFO]  Saved checkpoint to "imputer_model/model-0000.params"
2020-01-02 11:27:31,061 [INFO]  Epoch[0] Validation-cross-entropy=6.942389
2020-01-02 11:27:31,062 [INFO]  Epoch[0] Validation-0-accuracy=0.000000
2020-01-02 11:27:31,382 [INFO]  Epoch[1] Batch [0-465]	Speed: 23475.14 samples/sec	cross-entropy=19.875263	0-accuracy=0.000000
2020-01-02 11:27:31,700 [INFO]  Epoch[1] Train-cross-entropy=11.282785
2020-01-02 11:27:31,702 [INFO]  Epoch[1] Train-0-accuracy=0.000000
2020-01-02 11:27:31,703 [INFO]  Epoch[1] Time cost=0.64

CPU times: user 21.6 s, sys: 978 ms, total: 22.6 s
Wall time: 18.9 s


/Users/aureliegiraud/.virtualenvs/ebr/lib/python3.7/site-packages/datawig/imputer.py:822: FutureWarning: The 'contains' method is deprecated and will be removed in a future version. Use 'key in index' instead of 'index.contains(key)'
  if data_frame.columns.contains(imputation_col):


,0,1,2,3,4,5,6,7,target,0_imputed
15801,NaN,30.0,4.467954,1.062335,1935.0,1.698859,37.76,-122.44,3.86100,3.904469
1903,NaN,15.0,11.679878,1.905488,984.0,3.000000,38.91,-119.92,1.62500,4.225455
10336,7.1193,2.0,9.093750,1.093750,199.0,3.109375,33.81,-117.76,5.00001,8.412660
16077,NaN,52.0,5.495164,1.000000,1372.0,2.653772,37.74,-122.48,3.35000,4.393540
9565,NaN,11.0,4.140609,1.026233,2814.0,2.952781,37.36,-120.59,0.87300,2.592965
...,...,...,...,...,...,...,...,...,...,...
9372,NaN,37.0,5.109792,1.044510,648.0,1.922849,37.93,-122.53,3.10300,4.165971
7291,1.2059,18.0,2.537037,1.089744,1913.0,2.725071,33.98,-118.22,2.55000,2.423422
17728,5.5336,6.0,4.905336,0.965577,2160.0,3.717728,37.32,-121.79,2.41900,4.806883
7293,2.2059,35.0,2.748491,0.997988,2160.0,4.346076,33.98,-118.23,1.50000,2.346575
